# Artificial Neural Network 1

In [ ]:
#Importando librearias
import sys
import os
import json
import mlflow
import mlflow.keras
import tempfile
import pandas as pd
import seaborn as sns
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
sys.path.append('../')
sys.path.append('../../')
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.regularizers import l2
from kerastuner import HyperModel
from kerastuner.tuners import Hyperband
import Python.Private.constants as constants
from Python.Style.styles import  *
from scipy.stats import expon, randint
from sklearn.model_selection import RandomizedSearchCV, GroupKFold
from sklearn.metrics import make_scorer, accuracy_score, f1_score, confusion_matrix

In [ ]:
#Desactivando wrnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Leyendo datos
#training = spark.read.parquet('/mnt/Data/DataMart/Views/TrainingViews/training_shuffled.parquet')
#training_us = spark.read.parquet('/mnt/Data/DataMart/Views/TrainingViews/undersampled_shuffled.parquet')
#training_os = spark.read.parquet('/mnt/Data/DataMart/Views/TrainingViews/oversampled_shuffled.parquet')
#testing1 = spark.read.parquet('/mnt/Data/DataMart/Views/TrainingViews/testing1.parquet')
#testing2 = spark.read.parquet('/mnt/Data/DataMart/Views/TrainingViews/testing2.parquet')

In [ ]:
#training.write.format("delta").saveAsTable("training")
#training_us.write.format("delta").saveAsTable("training_us")
#training_os.write.format("delta").saveAsTable("training_os")
#testing1.write.format("delta").saveAsTable("testing1")
#testing2.write.format("delta").saveAsTable("testing2")

In [ ]:
#Leyendo datos
training = spark.sql("SELECT * FROM training")
training_us = spark.sql("SELECT * FROM training_us")
training_os = spark.sql("SELECT * FROM training_os")
testing1 = spark.sql("SELECT * FROM testing1")
testing2 = spark.sql("SELECT * FROM testing2")

In [ ]:
#To delete columns
total_columns = training.columns
tdc = ['serialNumber','serialNumber_neighbor','fixed_path','FE-Comments','Atenna','Conjunto','PSNumber']
training_cols = list(filter(lambda x: x not in tdc, total_columns))
#Validation datasets
sn_unique_training = training.select("serialNumber").distinct()
sn_unique_training_us = training_us.select("serialNumber").distinct()
sn_unique_training_os = training_os.select("serialNumber").distinct()
#Dividiendo por serialNumber
train_sn_t, train_sn_v = sn_unique_training.randomSplit([0.8,0.2], seed = 1010)
train_us_sn_t, train_us_sn_v = sn_unique_training_us.randomSplit([0.8,0.2], seed = 1010)
train_os_sn_t, train_os_sn_v = sn_unique_training_os.randomSplit([0.8,0.2], seed = 1010)
#Seleccionando por serialNumber
train_t = training.join(train_sn_t, on = ["serialNumber"], how = "inner")
train_v = training.join(train_sn_v, on = ["serialNumber"], how = "inner")
train_us_t = training_us.join(train_us_sn_t, on = ["serialNumber"], how = "inner")
train_us_v = training_us.join(train_us_sn_v, on = ["serialNumber"], how = "inner")
train_os_t = training_os.join(train_os_sn_t, on = ["serialNumber"], how = "inner")
train_os_v = training_os.join(train_os_sn_v, on = ["serialNumber"], how = "inner")

In [ ]:
#Validando etiquetas en conjuntos de datos
print("train_t")
count_train_t = train_t.groupBy("Communicating").count()
count_train_t.show()
print("train_v")
count_train_v = train_v.groupBy("Communicating").count()
count_train_v.show()
print("train_us_t")
count_train_us_t = train_us_t.groupBy("Communicating").count()
count_train_us_t.show()
print("train_us_v")
count_train_us_v = train_us_v.groupBy("Communicating").count()
count_train_us_v.show()
print("train_os_t")
count_train_os_t = train_os_t.groupBy("Communicating").count()
count_train_os_t.show()
print("train_os_v")
count_train_os_v = train_os_v.groupBy("Communicating").count()
count_train_os_v.show()


train_t
+-------------+------+
|Communicating| count|
+-------------+------+
|            0| 20749|
|            1|495191|
+-------------+------+

train_v
+-------------+------+
|Communicating| count|
+-------------+------+
|            0|  5132|
|            1|116914|
+-------------+------+

train_us_t
+-------------+-----+
|Communicating|count|
+-------------+-----+
|            0|19533|
|            1|21047|
+-------------+-----+

train_us_v
+-------------+-----+
|Communicating|count|
+-------------+-----+
|            0| 6348|
|            1| 4834|
+-------------+-----+

train_os_t
+-------------+------+
|Communicating| count|
+-------------+------+
|            0|490221|
|            1|495191|
+-------------+------+

train_os_v
+-------------+------+
|Communicating| count|
+-------------+------+
|            0|121884|
|            1|116914|
+-------------+------+



In [ ]:
#Seleccionando columnas
train_t = train_t.select(*training_cols)
train_v = train_v.select(*training_cols)
train_us_t = train_us_t.select(*training_cols)
train_us_v = train_us_v.select(*training_cols)
train_os_t = train_os_t.select(*training_cols)
train_os_v = train_os_v.select(*training_cols)
#Conjuntos de datos de prueba
testing1 = testing1.select(*training_cols)
testing2 = testing2.select(*training_cols)


In [ ]:
#Dividiendo datos training
train_t_y = train_t.select("Communicating")
train_t_x = train_t.drop("Communicating")
train_v_y = train_v.select("Communicating")
train_v_x = train_v.drop("Communicating")
#Dividiendo datos training undersampled
train_us_t_y = train_us_t.select("Communicating")
train_us_t_x = train_us_t.drop("Communicating")
train_us_v_y = train_us_v.select("Communicating")
train_us_v_x = train_us_v.drop("Communicating")
#Dividiendo datos training oversampled
train_os_t_y = train_os_t.select("Communicating")
train_os_t_x = train_os_t.drop("Communicating")
train_os_v_y = train_os_v.select("Communicating")
train_os_v_x = train_os_v.drop("Communicating")
#Dividiendo conjunto de datos de testeo
testing1_y = testing1.select("Communicating")
testing1_x = testing1.drop("Communicating")
testing2_y = testing2.select("Communicating")
testing2_x = testing2.drop("Communicating")

In [ ]:
#Conversion for General Training Data
train_t_y = train_t_y.toPandas().astype(np.float32).values
train_t_x = train_t_x.toPandas().astype(np.float32).values
train_v_y = train_v_y.toPandas().astype(np.float32).values
train_v_x = train_v_x.toPandas().astype(np.float32).values
#Conversion for Training Data - Undersampled
train_us_t_y = train_us_t_y.toPandas().astype(np.float32).values
train_us_t_x = train_us_t_x.toPandas().astype(np.float32).values
train_us_v_y = train_us_v_y.toPandas().astype(np.float32).values
train_us_v_x = train_us_v_x.toPandas().astype(np.float32).values
#Conversion for Training Data - Oversampled
train_os_t_y = train_os_t_y.toPandas().astype(np.float32).values
train_os_t_x = train_os_t_x.toPandas().astype(np.float32).values
train_os_v_y = train_os_v_y.toPandas().astype(np.float32).values
train_os_v_x = train_os_v_x.toPandas().astype(np.float32).values
#Conversión conjuntos de datos de testeo
testing1_y = testing1_y.toPandas().astype(np.float32).values
testing1_x = testing1_x.toPandas().astype(np.float32).values
testing2_y = testing2_y.toPandas().astype(np.float32).values
testing2_x = testing2_x.toPandas().astype(np.float32).values

In [ ]:
class MyHyperModel(HyperModel):
    def build(self, hp):
        inputs = Input(shape=(train_t_x.shape[1],))

        # Primera capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_1', min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_1', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(inputs)
        x = Dropout(hp.Float('dropout_1', min_value=0, max_value=0.5, step=0.1))(x)

        # Segunda capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(x)
        x = Dropout(hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1))(x)

        # Capa de salida
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='binary_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
#Tuner
tuner = Hyperband(
    MyHyperModel(),
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='dl_dir',
    project_name='keras_tuner_hyperband'
)

Reloading Tuner from dl_dir/keras_tuner_hyperband/tuner0.json


In [ ]:
mlflow.set_experiment('/Users/victor.hugo.borrayo@gmail.com/DL')

## Ajustando para undersampled

In [ ]:
#Umbrales de discretización
def find_threshold(model, x, y):
    continuous_pred = model.predict(x)
    thres = []
    f1_macro = []
    accuracy = []
    mpca = []
    for threshold in np.arange(0.1,1.0,0.01):
        thres.append(threshold)
        pred = (continuous_pred > threshold).astype(int)
        f1_macro.append(f1_score(y, pred, average='macro'))
        accuracy.append(accuracy_score(y, pred))
        cm = confusion_matrix(y, pred)
        #Balanced accuracy
        tp = cm[1,1]
        fp = cm[0,1]
        tn = cm[0,0]
        fn = cm[1,0]
        #Precisión por clase
        precision_class_0 = tn / (tn + fp)
        precision_class_1 = tp / (tp + fn)
        temp_mpca = (precision_class_0 + precision_class_1) / 2
        mpca.append(temp_mpca)
    #Encontrando máximos
    tres_f1 = thres[f1_macro.index(max(f1_macro))]
    tres_acc = thres[accuracy.index(max(accuracy))]
    thres_mpca = thres[mpca.index(max(mpca))]
    thresholds = {
        "f1_thres": tres_f1,
        "acc_thres": tres_acc,
        "mpca_thres": thres_mpca
    }
    return thresholds


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_us"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_undersampled")
    #Almacenando información de distribución
    tuner.search(
        train_us_t_x, train_us_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_us_v_x, train_us_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_us_v_x,train_us_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})    


2024/06/12 03:24:48 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/12 03:24:48 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-7f55d-6c0f4-e9e8d-a/tmp7g8d3zbf/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-7f55d-6c0f4-e9e8d-a/tmp7g8d3zbf/model/data/model/assets


350/350 [==============================] - 0s 923us/step


In [ ]:
tuner.results_summary()

Results summary
Results in dl_dir/keras_tuner_hyperband
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0008 summary
Hyperparameters:
units_1: 320
l2_1: 0.00019634218681491059
dropout_1: 0.2
units_2: 256
l2_2: 1.3587231266604919e-05
dropout_2: 0.2
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.5653729438781738

Trial 0018 summary
Hyperparameters:
units_1: 512
l2_1: 3.482061632108658e-05
dropout_1: 0.2
units_2: 160
l2_2: 9.322603482929009e-05
dropout_2: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.5162761807441711

Trial 0014 summary
Hyperparameters:
units_1: 128
l2_1: 0.009837687450413713
dropout_1: 0.30000000000000004
units_2: 32
l2_2: 9.268854650761784e-05
dropout_2: 0.1
learning_rate: 0.0001
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0001
Score: 0.5157395601272583

Trial 0009 summar

## Ajustando para oversampled

In [ ]:
#Tuner
tuner2 = Hyperband(
    MyHyperModel(),
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='keras_tuner_hyperband_2'
)

Reloading Tuner from my_dir/keras_tuner_hyperband_2/tuner0.json


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_os_1"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner2.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner2.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner2.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_os_v_x,train_os_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/16 02:50:16 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/16 02:50:16 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-1c0db-09384-6bb51-1/tmpws081ynn/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-1c0db-09384-6bb51-1/tmpws081ynn/model/data/model/assets


7463/7463 [==============================] - 6s 808us/step


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_os_1_extra_us"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner2.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner2.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner2.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_us_v_x,train_us_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/18 07:55:50 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/18 07:55:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpr54z2wpu/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpr54z2wpu/model/data/model/assets


350/350 [==============================] - 0s 829us/step


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_os_1_extra_t"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner2.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner2.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner2.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_v_x,train_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/18 08:08:46 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/18 08:08:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpuoovvhq8/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpuoovvhq8/model/data/model/assets


3814/3814 [==============================] - 3s 852us/step


In [ ]:
tuner2.results_summary()

Results summary
Results in my_dir/keras_tuner_hyperband_2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0012 summary
Hyperparameters:
units_1: 320
l2_1: 0.0037291732451233354
dropout_1: 0.4
units_2: 32
l2_2: 0.0017500032927377685
dropout_2: 0.4
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0005
Score: 0.7832268476486206

Trial 0024 summary
Hyperparameters:
units_1: 352
l2_1: 0.0036619880519683226
dropout_1: 0.4
units_2: 64
l2_2: 0.0005541713268450252
dropout_2: 0.30000000000000004
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0021
Score: 0.7774227857589722

Trial 0005 summary
Hyperparameters:
units_1: 320
l2_1: 0.0037291732451233354
dropout_1: 0.4
units_2: 32
l2_2: 0.0017500032927377685
dropout_2: 0.4
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7745332717895508

Trial 0026 su

## Ajustando para oversampled 2

In [ ]:
class MyHyperModel2(HyperModel):
    def build(self, hp):
        inputs = Input(shape=(train_t_x.shape[1],))

        # Primera capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_1', min_value=32, max_value=1024, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_1', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(inputs)
        x = Dropout(hp.Float('dropout_1', min_value=0, max_value=0.5, step=0.1))(x)

        # Segunda capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_2', min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(x)
        x = Dropout(hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1))(x)

        # Tercera capa densa con regularización L2
        x = Dense(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            activation='relu',
            kernel_regularizer=l2(hp.Float('l2_2', min_value=1e-5, max_value=1e-2, sampling='LOG'))
        )(x)
        x = Dropout(hp.Float('dropout_2', min_value=0, max_value=0.5, step=0.1))(x)


        # Capa de salida
        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
            loss='binary_crossentropy',
            metrics=['accuracy'])

        return model

In [ ]:
#Tuner
tuner3 = Hyperband(
    MyHyperModel2(),
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='dl_dir',
    project_name='keras_tuner_hyperband_dl_os_2'
)

Reloading Tuner from dl_dir/keras_tuner_hyperband_dl_os_2/tuner0.json


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_os_2"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner3.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner3.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner3.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_os_v_x,train_os_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/16 02:58:47 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/16 02:58:47 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-1c0db-09384-6bb51-1/tmpy2cb0qji/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-1c0db-09384-6bb51-1/tmpy2cb0qji/model/data/model/assets


7463/7463 [==============================] - 7s 938us/step


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_os_2_extra_us"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner3.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner3.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner3.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_us_v_x,train_us_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/18 08:14:55 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/18 08:14:55 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpd3a7muxv/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpd3a7muxv/model/data/model/assets


350/350 [==============================] - 0s 1ms/step


In [ ]:
#Ajustando red neuronal
with mlflow.start_run(run_name="ann_tunning_os_2_extra_t"):
    mlflow.log_param("model","ann")
    mlflow.log_param("data","training_oversampled")
    #Almacenando información de distribución
    tuner3.search(
        train_os_t_x, train_os_t_y,  # Conjunto de datos de entrenamiento
        validation_data=(train_os_v_x, train_os_v_y),  # Conjunto de datos de validación
        epochs=10,
        batch_size=32
    )
    #Obteniendo mejor modelo
    best_model = tuner3.get_best_models(num_models=1)[0]
    #Guardando modelo
    mlflow.keras.log_model(best_model, "best_model")
    #Obtener el mejor conjunto de hiperparámetros
    best_hyperparameters = tuner3.get_best_hyperparameters(num_trials=1)[0]
    #Guardando parametros
    params = {}
    for param, value in best_hyperparameters.values.items():
        params[param] = value
    mlflow.log_params(params)
    #Obteniendo predicciones
    y_pred_t1 = best_model.predict(testing1_x)
    y_pred_t2 = best_model.predict(testing2_x)
    #Discretizando
    y_pred_t1 = (y_pred_t1 > 0.5).astype(int)
    y_pred_t2 = (y_pred_t2 > 0.5).astype(int)
    #Umbrales de discretización
    thresholds = find_threshold(best_model, train_v_x,train_v_y)
    mlflow.log_params(thresholds)
    #Almacenando métricas con conjunto de datos
    f1_test1 = f1_score(testing1_y, y_pred_t1, average='weighted')
    f1_test2 = f1_score(testing2_y, y_pred_t2, average='weighted')
    accuracy_test1 = accuracy_score(testing1_y, y_pred_t1)
    accuracy_test2 = accuracy_score(testing2_y, y_pred_t2)
    mlflow.log_metrics({"f1_test1":f1_test1, "f1_test2":f1_test2, "accuracy_test1":accuracy_test1, "accuracy_test2":accuracy_test2})

2024/06/18 08:18:35 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.10.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
2024/06/18 08:18:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpqyu31n89/model/data/model/assets


INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-53e1c-ed80e-10538/tmpqyu31n89/model/data/model/assets


3814/3814 [==============================] - 4s 1ms/step


In [ ]:
tuner3.results_summary()

Results summary
Results in dl_dir/keras_tuner_hyperband_dl_os_2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0016 summary
Hyperparameters:
units_1: 352
l2_1: 0.0047827931010667075
dropout_1: 0.4
units_2: 160
l2_2: 0.00027523785246150276
dropout_2: 0.1
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 0.7868030667304993

Trial 0026 summary
Hyperparameters:
units_1: 512
l2_1: 6.544879891698117e-05
dropout_1: 0.30000000000000004
units_2: 320
l2_2: 5.3132680509292734e-05
dropout_2: 0.4
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7780885696411133

Trial 0000 summary
Hyperparameters:
units_1: 1024
l2_1: 4.875586071674114e-05
dropout_1: 0.2
units_2: 512
l2_2: 0.002194899412113859
dropout_2: 0.1
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0
Score: 0.7766815423965454

Trial 0013 summary
Hyper